In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/testGAN

/content/drive/My Drive/testGAN


In [3]:
! pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 13.3MB/s 


In [4]:
import torch
import torch.nn as nn

import spacy
import numpy as np
import pandas as pd

import random
import math
import time

from BERT_WordEmbeddingsPipeline import embeddingsPipeline
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore") 

SEED = 25

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

100%|██████████| 407873900/407873900 [00:41<00:00, 9931739.76B/s]


In [0]:
data = pd.read_csv('hpl.csv')
data = data[:3500]

embedding_length = 768
max_words = 100

zero_embedding = [0 for i in range(embedding_length)]

emb = []
for i in data['tokenized_sents']:
    e = embeddingsPipeline(i)
    while(len(e) < max_words):
        e.append(zero_embedding)
    e = e[:max_words]
    emb.append(e)
data['embeddings'] = emb

train_data = data

In [0]:
data = pd.read_csv('eap.csv')
data = data[:100]

embedding_length = 768
max_words = 100

zero_embedding = [0 for i in range(embedding_length)]

emb = []
for i in data['tokenized_sents']:
    e = embeddingsPipeline(i)
    while(len(e) < max_words):
        e.append(zero_embedding)
    e = e[:max_words]
    emb.append(e)
data['embeddings'] = emb

test_data = data

In [7]:
print(train_data.shape)
print(test_data.shape)

(3500, 2)
(100, 2)


In [0]:
INPUT_SIZE = embedding_length
HIDDEN_SIZE = 512
NUM_LAYERS = 2
OUTPUT_SIZE = embedding_length
BATCH_SIZE = 32

In [0]:
train_emb = torch.Tensor(train_data['embeddings'])
train_dataSet = torch.utils.data.TensorDataset(train_emb)
train_dataLoader = torch.utils.data.DataLoader(train_dataSet, batch_size=BATCH_SIZE, drop_last=True)

In [0]:
test_emb = torch.Tensor(test_data['embeddings'])
test_dataSet = torch.utils.data.TensorDataset(test_emb)
test_dataLoader = torch.utils.data.DataLoader(test_dataSet, batch_size=BATCH_SIZE, drop_last=True)

In [0]:
from Encoder import Encoder
from Decoder import Decoder
from noTeacherForcingGenerator import Generator
from Discriminator import Discriminator

In [12]:
enc = Encoder(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS)
dec = Decoder(HIDDEN_SIZE, OUTPUT_SIZE, NUM_LAYERS)
# gen = Generator(enc, dec, max_words, HIDDEN_SIZE, embedding_length)
gen = Generator(enc, dec)
print(gen)

dis = Discriminator(INPUT_SIZE, NUM_LAYERS, max_words)
print(dis)

Generator(
  (encoder): Encoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (lstm): LSTM(768, 512, num_layers=2, batch_first=True, bidirectional=True)
    (fc1): Linear(in_features=1024, out_features=512, bias=True)
    (lrelu): LeakyReLU(negative_slope=0.01)
  )
  (decoder): Decoder(
    (lstm): LSTM(512, 768, num_layers=2, batch_first=True, bidirectional=True)
    (fc1): Linear(in_features=1536, out_features=768, bias=True)
  )
)
Discriminator(
  (lstm): LSTM(768, 1, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=100, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [0]:
def train(dataLoader, test_dataLoader, gen, dis, num_epochs, max_words, batch_size, embedding_length):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    gen.to(device)
    dis.to(device)
    
    criterion = nn.BCELoss()
    d_optimizer = torch.optim.Adam(dis.parameters(), lr=0.0002)
    g_optimizer = torch.optim.Adadelta(gen.parameters(), lr=0.0004)
    
    for epoch in range(num_epochs):
        start_time = time.time()

        dis.train()
        gen.train()
        
        real_labels = torch.ones(1, batch_size).to(device)
        fake_labels = torch.zeros(1, batch_size).to(device)
        
        #---------------------------------------------------#
        # Training Discriminator
        #---------------------------------------------------#
        total_discriminator_loss = 0
        iterator = iter(dataLoader)
        
        for i in range(len(dataLoader)):
            batch_data = next(iterator) 
            # batch_data is a one-element list containing a single batch in the form of Tensor.
            batch = batch_data[0]
            
            batch = batch.to(device)
            
            dis_output = torch.zeros(batch_size)
            for i, embedding in enumerate(batch):
                dis_output[i] = dis(embedding.view(1, max_words, embedding_length))
            
            dis_output = dis_output.to(device)
            d_loss_real = criterion(dis_output, real_labels)
            
            fake_batch = gen(batch)
            fake_batch = fake_batch.to(device)
            
            dis_output = torch.zeros(batch_size)
            for i, embedding in enumerate(fake_batch):
                dis_output[i] = dis(embedding.view(1, max_words, embedding_length))
                
            dis_output = dis_output.to(device)
            d_loss_fake = criterion(dis_output, fake_labels)
            
            d_loss = d_loss_real + d_loss_fake
            total_discriminator_loss += d_loss.item()
            
            d_optimizer.zero_grad()
            d_loss.backward()
            d_optimizer.step()
    
        #---------------------------------------------------#
        # Training Generator
        #---------------------------------------------------#
        total_generator_loss = 0
        iterator = iter(dataLoader)
        
        for i in range(len(dataLoader)):
            batch_data = next(iterator) 
            # batch_data is a one-element list containing a single batch in the form of Tensor.
            batch = batch_data[0]
            batch = batch.to(device)
            
            generated_batch = gen(batch)
            generated_batch = generated_batch.to(device)
            
            dis_output = torch.zeros(batch_size)
            for i, embedding in enumerate(generated_batch):
                dis_output[i] = dis(embedding.view(1, max_words, embedding_length))
            
            dis_output = dis_output.to(device)
            g_loss = criterion(dis_output, real_labels)
            total_generator_loss += g_loss.item()
            
            g_optimizer.zero_grad()
            g_loss.backward()
            g_optimizer.step()
            
        
        #---------------------------------------------------#
        # On Test data 
        #---------------------------------------------------#
        dis.eval()
        gen.eval()
        
        generator_loss_on_test = 0
        discriminator_loss_on_test = 0
        iterator = iter(test_dataLoader)
        
        for i in range(len(test_dataLoader)):
            batch_data = next(iterator) 
            # batch_data is a one-element list containing a single batch in the form of Tensor.
            batch = batch_data[0]
            batch = batch.to(device)
            
            dis_output = torch.zeros(batch_size)
            for i, embedding in enumerate(batch):
                dis_output[i] = dis(embedding.view(1, max_words, embedding_length))
            
            dis_output = dis_output.to(device)
            d_loss = criterion(dis_output, real_labels)
            discriminator_loss_on_test += d_loss.item()

            generated_batch = gen(batch)
            generated_batch = generated_batch.to(device)
            dis_output = torch.zeros(batch_size)
            
            for i, embedding in enumerate(generated_batch):
                dis_output[i] = dis(embedding.view(1, max_words, embedding_length))
            
            dis_output = dis_output.to(device)
            g_loss = criterion(dis_output, real_labels)
            d_loss = criterion(dis_output, fake_labels)

            discriminator_loss_on_test += d_loss.item()
            generator_loss_on_test += g_loss.item()
            
        
        finish_time = time.time()
        avg_d_loss = total_discriminator_loss/len(dataLoader)
        avg_g_loss = total_generator_loss/len(dataLoader)
        d_loss_test = discriminator_loss_on_test/len(test_dataLoader)
        g_loss_test = generator_loss_on_test/len(test_dataLoader)
        time_taken = finish_time-start_time
        
        print("Epoch[{:02}/{}]: avg_d_Loss: {:.4f}, avg_g_Loss: {:.4f}, d_loss_test: {:.4f}, g_loss_test: {:.4f}, Time : {:.2f}" 
             .format(epoch+1, num_epochs, avg_d_loss, avg_g_loss, d_loss_test, g_loss_test, time_taken))

In [14]:
train(train_dataLoader, test_dataLoader, gen, dis, 10, max_words, BATCH_SIZE, embedding_length)

Epoch[01/10]: avg_d_Loss: 1.3741, avg_g_Loss: 0.7248, d_loss_test: 1.3586, g_loss_test: 0.7246, Time : 109.36
Epoch[02/10]: avg_d_Loss: 1.3182, avg_g_Loss: 0.7800, d_loss_test: 1.2804, g_loss_test: 0.7786, Time : 111.97
Epoch[03/10]: avg_d_Loss: 1.1840, avg_g_Loss: 0.8890, d_loss_test: 1.1338, g_loss_test: 0.8828, Time : 112.58
Epoch[04/10]: avg_d_Loss: 0.9925, avg_g_Loss: 1.0314, d_loss_test: 0.9765, g_loss_test: 1.0122, Time : 113.17
Epoch[05/10]: avg_d_Loss: 0.8112, avg_g_Loss: 1.1693, d_loss_test: 0.8450, g_loss_test: 1.1367, Time : 115.79
Epoch[06/10]: avg_d_Loss: 0.6723, avg_g_Loss: 1.2962, d_loss_test: 0.7375, g_loss_test: 1.2699, Time : 115.09
Epoch[07/10]: avg_d_Loss: 0.5620, avg_g_Loss: 1.4378, d_loss_test: 0.6421, g_loss_test: 1.4237, Time : 114.09
Epoch[08/10]: avg_d_Loss: 0.4677, avg_g_Loss: 1.6035, d_loss_test: 0.5548, g_loss_test: 1.5964, Time : 114.30
Epoch[09/10]: avg_d_Loss: 0.3874, avg_g_Loss: 1.7840, d_loss_test: 0.4800, g_loss_test: 1.7803, Time : 114.02
Epoch[10/1

In [0]:
def additional_generator_training(dataLoader, test_dataLoader, gen, dis, num_epochs, max_words, batch_size, embedding_length):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    gen.to(device)
    dis.to(device)
    
    criterion = nn.BCELoss()
    g_optimizer = torch.optim.Adadelta(gen.parameters(), lr=0.0004)
    
    for epoch in range(num_epochs):
        start_time = time.time()

        dis.train()
        gen.train()
        
        real_labels = torch.ones(1, batch_size).to(device)
        fake_labels = torch.zeros(1, batch_size).to(device)
        
        #---------------------------------------------------#
        # Loss from Discriminator on training data
        #---------------------------------------------------#
        total_discriminator_loss = 0
        iterator = iter(dataLoader)
        
        for i in range(len(dataLoader)):
            batch_data = next(iterator) 
            # batch_data is a one-element list containing a single batch in the form of Tensor.
            batch = batch_data[0]
            
            batch = batch.to(device)
            
            dis_output = torch.zeros(batch_size)
            for i, embedding in enumerate(batch):
                dis_output[i] = dis(embedding.view(1, max_words, embedding_length))
            
            dis_output = dis_output.to(device)
            d_loss_real = criterion(dis_output, real_labels)
            
            fake_batch = gen(batch)
            fake_batch = fake_batch.to(device)
            
            dis_output = torch.zeros(batch_size)
            for i, embedding in enumerate(fake_batch):
                dis_output[i] = dis(embedding.view(1, max_words, embedding_length))
                
            dis_output = dis_output.to(device)
            d_loss_fake = criterion(dis_output, fake_labels)
            
            d_loss = d_loss_real + d_loss_fake
            total_discriminator_loss += d_loss.item()
        
        #---------------------------------------------------#
        # Training Generator
        #---------------------------------------------------#
        total_generator_loss = 0
        iterator = iter(dataLoader)
        
        for i in range(len(dataLoader)):
            batch_data = next(iterator) 
            # batch_data is a one-element list containing a single batch in the form of Tensor.
            batch = batch_data[0]
            batch = batch.to(device)
            
            generated_batch = gen(batch)
            generated_batch = generated_batch.to(device)
            
            dis_output = torch.zeros(batch_size)
            for i, embedding in enumerate(generated_batch):
                dis_output[i] = dis(embedding.view(1, max_words, embedding_length))
            
            dis_output = dis_output.to(device)
            g_loss = criterion(dis_output, real_labels)
            total_generator_loss += g_loss.item()
            
            g_optimizer.zero_grad()
            g_loss.backward()
            g_optimizer.step()
            
            
        #---------------------------------------------------#
        # On Test data 
        #---------------------------------------------------#
        dis.eval()
        gen.eval()
        
        generator_loss_on_test = 0
        discriminator_loss_on_test = 0
        iterator = iter(test_dataLoader)
        
        for i in range(len(test_dataLoader)):
            batch_data = next(iterator) 
            # batch_data is a one-element list containing a single batch in the form of Tensor.
            batch = batch_data[0]
            batch = batch.to(device)
            
            dis_output = torch.zeros(batch_size)
            for i, embedding in enumerate(batch):
                dis_output[i] = dis(embedding.view(1, max_words, embedding_length))
            
            dis_output = dis_output.to(device)
            d_loss_real = criterion(dis_output, real_labels)

            generated_batch = gen(batch)
            generated_batch = generated_batch.to(device)
            
            dis_output = torch.zeros(batch_size)
            for i, embedding in enumerate(generated_batch):
                dis_output[i] = dis(embedding.view(1, max_words, embedding_length))
            
            dis_output = dis_output.to(device)
            d_loss_fake = criterion(dis_output, fake_labels)
            
            d_loss = d_loss_real + d_loss_fake
            discriminator_loss_on_test += d_loss.item()
            
            generated_batch = gen(batch)
            generated_batch = generated_batch.to(device)
            dis_output = torch.zeros(batch_size)
            
            for i, embedding in enumerate(generated_batch):
                dis_output[i] = dis(embedding.view(1, max_words, embedding_length))
            
            dis_output = dis_output.to(device)
            g_loss = criterion(dis_output, real_labels)
            generator_loss_on_test += g_loss.item()
            
            
        finish_time = time.time()
        avg_d_loss = total_discriminator_loss/len(dataLoader)
        avg_g_loss = total_generator_loss/len(dataLoader)
        d_loss_test = discriminator_loss_on_test/len(test_dataLoader)
        g_loss_test = generator_loss_on_test/len(test_dataLoader)
        time_taken = finish_time-start_time
        
        print("Epoch[{:02}/{:02}]: avg_d_Loss: {:.4f}, avg_g_Loss: {:.6f}, d_loss_test: {:.6f}, g_loss_test: {:.6f}, Time : {:.2f}" 
             .format(epoch+1, num_epochs, avg_d_loss, avg_g_loss, d_loss_test, g_loss_test, time_taken))

In [16]:
additional_generator_training(train_dataLoader, test_dataLoader, gen, dis, 10, max_words, BATCH_SIZE, embedding_length)

Epoch[01/10]: avg_d_Loss: 0.2923, avg_g_Loss: 1.963665, d_loss_test: 0.416645, g_loss_test: 1.963344, Time : 79.21
Epoch[02/10]: avg_d_Loss: 0.2929, avg_g_Loss: 1.960535, d_loss_test: 0.417112, g_loss_test: 1.960481, Time : 79.63
Epoch[03/10]: avg_d_Loss: 0.2933, avg_g_Loss: 1.958062, d_loss_test: 0.417501, g_loss_test: 1.958107, Time : 79.08
Epoch[04/10]: avg_d_Loss: 0.2937, avg_g_Loss: 1.955875, d_loss_test: 0.417871, g_loss_test: 1.955852, Time : 79.25
Epoch[05/10]: avg_d_Loss: 0.2941, avg_g_Loss: 1.953642, d_loss_test: 0.418273, g_loss_test: 1.953412, Time : 79.40
Epoch[06/10]: avg_d_Loss: 0.2945, avg_g_Loss: 1.951079, d_loss_test: 0.418763, g_loss_test: 1.950448, Time : 79.00
Epoch[07/10]: avg_d_Loss: 0.2949, avg_g_Loss: 1.947733, d_loss_test: 0.419449, g_loss_test: 1.946319, Time : 79.45
Epoch[08/10]: avg_d_Loss: 0.2956, avg_g_Loss: 1.942471, d_loss_test: 0.420671, g_loss_test: 1.939009, Time : 79.29
Epoch[09/10]: avg_d_Loss: 0.2968, avg_g_Loss: 1.930747, d_loss_test: 0.424064, g